In [17]:
from moviepy.editor import VideoFileClip
import os
import datetime
from docx import Document
import openai
import os
import tempfile
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
from email.mime.text import MIMEText
#Placez la clé API openai ici
openai.api_key = ''

In [10]:
def transcribe_video(video_path):
    video = VideoFileClip(video_path)
    audio = video.audio
    temp_audio_fd, temp_audio_path = tempfile.mkstemp(suffix=".wav")
    os.close(temp_audio_fd)
    try:
        audio.write_audiofile(temp_audio_path, codec='pcm_s16le')
        with open(temp_audio_path, "rb") as audio_file:
            response = openai.audio.transcriptions.create(
                file=audio_file,
                model="whisper-1",
            )
        text = response.text
    finally:
        os.remove(temp_audio_path)
    return "S'il vous plaît, aidez-moi à résumer le paragraphe suivant: " + text

In [11]:
def summarize_text_with_chatgpt(text):
    response = openai.completions.create(
        model="gpt-3.5-turbo-instruct",
        prompt=text,
        temperature=0.6,
        max_tokens=2000
    )
    summary = response.choices[0].text.strip()
    return summary

In [12]:
def save_summary_as_word(summary, filename):
    doc = Document()
    doc.add_paragraph(summary)
    doc.save(filename)

In [26]:
def email(doc, address_email):
    
    mail_server = "smtp.gmail.com"
    #Veuillez remplacer le nom d'utilisateur par votre propre adresse e-mail
    mail_username = "yezhanyz@gmail.com"
    #Veuillez remplacer le mot de passe par votre propre mot de passe
    mail_password = ""

    
    msg = MIMEMultipart()
    msg['From'] = mail_username
    msg['To'] = address_email
    msg['Subject'] = "Résumé de votre réunion - À consulter"

    body =  """J'espère que cette journée vous trouve en pleine forme. Je vous envoie le résumé de votre réunion pour votre référence et vos actions ultérieures. Veuillez trouver ci-joint le document.

Je vous souhaite une journée agréable et productive.

Cordialement,

L'équipe du projet SayIt"""
    msg.attach(MIMEText(body, 'plain'))


    with open(doc, 'rb') as file:
        part = MIMEBase('application', 'octet-stream')
        part.set_payload(file.read())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition', f'attachment; filename= {os.path.basename(doc)}')
        msg.attach(part)

    server = smtplib.SMTP(mail_server, 587)
    server.starttls()
    server.login(mail_username, mail_password)
    server.send_message(msg)
    server.quit()

In [20]:
def main(video_path, address_email):
    current_dir = os.getcwd()
    file_type='docx'
    if video_path.endswith('.mp4'):
        text = transcribe_video(video_path)
        summary = summarize_text_with_chatgpt(text) 
        text_dir = os.path.join(current_dir, file_type)
        if not os.path.exists(text_dir):
            os.makedirs(text_dir)
        timestamp = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
        text_filename = os.path.basename(video_path).replace('.mp4', f'_{timestamp}.{file_type}')
        save_summary_as_word(summary, os.path.join(text_dir, text_filename))
        email(os.path.join(text_dir, text_filename), address_email)
    else:
        print("The provided file is not an mp4 file.")

In [21]:
#Il est préférable d'utiliser votre propre adresse e-mail ici. Bien sûr, cela ne me dérange pas si vous utilisez la mienne. XD
main('./video/video.mp4', 'zhan.ye@edu.ece.fr')

MoviePy - Writing audio in C:\Users\14728\AppData\Local\Temp\tmpkm5b93dh.wav


MoviePy - Done.
